# Modifications de résultats des sondages

## Import préliminaires des librairies

In [7]:
# Command OS
import os

# File glob
import glob

# Regex
import re


# Calcul and data manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

# Dealing with specific representation of number
# Aka commat in french in numbers
import locale
from locale import atof





## Calcul de l'erreur à 95% de confiance

In [2]:
def error_p(p,n=1000,Za=1.96):
    return np.sqrt(p*(1-p)/n)*Za
error_p(0.48,300)

0.05653504399927535

## Calcul de l'intervalle de confiance

In [3]:
def margin_p(p,n=1000,Za=1.96):
    er=error_p(p,n,Za)
    return [p-er,p+er]


In [4]:
def error_in_margin(p,n,variation,Za=1.96):
    # Compute old values
    old_p=p-variation
    # Compute err
    err=error_p(p/100,n,Za)*100
    # Compute err old values
    old_err=error_p(old_p/100,n,Za)*100
    return (np.abs(variation)<=err+old_err) & variation!=0

In [12]:
df=None
for filep in glob.glob('data/*'):
    if df is None:
        df=pd.read_pickle(filep)
    else:
        df=pd.concat([df,pd.read_pickle(filep)],ignore_index=True)



In [18]:
df.describe()

,Base,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable
count,50784.000000,50784.000000,50784.000000,50784.000000,50784.000000,50784.00000,50784.000000,50784.000000,50784.000000,50784.000000
mean,228.643273,4.453186,22.145353,26.598876,25.368732,25.17846,50.547426,22.853972,0.064794,0.238382
std,233.991962,5.168782,11.626130,14.544700,7.807523,12.48890,14.648863,14.960845,8.250689,9.066258
min,4.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,-96.500000,-100.000000
25%,119.000000,1.500000,13.900000,16.400000,20.800000,16.80000,40.800000,10.600000,-3.600000,-4.100000
50%,165.000000,3.100000,20.200000,23.900000,25.400000,23.10000,49.900000,20.200000,0.000000,0.100000
75%,275.000000,5.700000,28.400000,33.600000,29.700000,31.30000,59.900000,33.100000,3.600000,4.600000
max,4100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,84.400000


## Verification pas de données bizarre

In [108]:
df[np.abs(df['Très favorable']+df['Plutôt favorable']-df['ST Favorable'])>0.11]

,Nom,Categorie,Groupe,Base,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable


In [109]:
df[np.abs(df['Très défavorable']+df['Plutôt défavorable']-df['ST Défavorable'])>0.11]

,Nom,Categorie,Groupe,Base,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable


In [110]:
df[np.abs(df['ST Favorable']+df['ST Défavorable']+df['Nsp'])<99.89]

,Nom,Categorie,Groupe,Base,Très favorable,Plutôt favorable,ST Favorable,Plutôt défavorable,Très défavorable,ST Défavorable,Nsp,Evolution ST Favorable,Evolution ST Défavorable


In [111]:
x=2081
df.iloc[x]

Nom                         Jean-Luc MELENCHON
Categorie                   Proximitépolitique
Groupe                                     UDI
Base                                        19
Très favorable                            22.2
Plutôt favorable                          66.2
ST Favorable                              88.3
Plutôt défavorable                           0
Très défavorable                             0
ST Défavorable                               0
Nsp                                       11.7
Evolution ST Favorable                       0
Evolution ST Défavorable                 -11.7
Name: 2081, dtype: object

In [112]:
df[df.Base==0]

unmatch=df.iloc[x]
# unmatch=df[np.abs(df['Très favorable']+df['Plutôt favorable']-df['ST Favorable'])>0.11].iloc[0]
# unmatch=df[np.abs(df['Très défavorable']+df['Plutôt défavorable']-df['ST Défavorable'])>0.11]
# unmatch=df[np.abs(df['ST Favorable']+df['ST Défavorable']+df['Nsp'])<99.89].iloc[0]

# id_unmatch=unmatch.index[0]
id_unmatch=x
line=unmatch
# line['Base']=4
# line['Très favorable']=0
# line['Plutôt favorable']=0
# line['ST Favorable']=0
# line['Plutôt défavorable']=35.5
# line['Très défavorable']=28.7
# line['ST Défavorable']=64.2
# line['Nsp']=0
# line['Evolution ST Favorable']=-39.6
line[['Plutôt défavorable','Très défavorable','ST Défavorable']]=line[['Très favorable','Plutôt favorable','ST Favorable']]
line[['Très favorable','Plutôt favorable','ST Favorable']]=0
print(id_unmatch)

line

2081


Nom                         Jean-Luc MELENCHON
Categorie                   Proximitépolitique
Groupe                                     UDI
Base                                        19
Très favorable                               0
Plutôt favorable                             0
ST Favorable                                 0
Plutôt défavorable                        22.2
Très défavorable                          66.2
ST Défavorable                            88.3
Nsp                                       11.7
Evolution ST Favorable                       0
Evolution ST Défavorable                 -11.7
Name: 2081, dtype: object

In [113]:
# df.loc[id_unmatch]=line

In [114]:
# df.to_pickle('data/'+filename+'.p')